In [1]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Chargement des données
data = pd.read_csv('iot23_18column.csv')
data = data.dropna()

# Séparation des instances bénignes du reste de l'ensemble de données
benign_data = data[data['label'] == 0]  # Supposons que le label 0 représente les instances bénignes
X_benign = benign_data.drop(columns=['label'])

# Standardisation des caractéristiques
scaler = StandardScaler()
X_benign_scaled = scaler.fit_transform(X_benign)

# Entraînement du modèle Isolation Forest sur les instances bénignes
isolation_forest = IsolationForest(contamination='auto', random_state=42)  
# Vous devez ajuster ces paramètres en fonction de vos données
isolation_forest.fit(X_benign_scaled)

# Application du modèle Isolation Forest à l'ensemble de données complet
X_all = data.drop(columns=['label'])
X_all_scaled = scaler.transform(X_all)
anomaly_scores = isolation_forest.decision_function(X_all_scaled)

# Extraction des instances prédites comme anomalies (scores négatifs)
predicted_anomalies = data[anomaly_scores < 0]

# Affichage des anomalies prédites
print("Anomalies prédites:", predicted_anomalies)
print("Nombre d'anomalies détectées:", len(predicted_anomalies))

Anomalies prédites:          Unnamed: 0      ts      uid  id.orig_h  id.orig_p  id.resp_h  \
3                 3       6   961436       1360    34243.0     205149   
28               28      49   971496       1360    48469.0     755468   
29               29      51   641351       1360    55730.0     907845   
30               30      54   489667       1360    50261.0     902004   
41               41      72   496373       1360    58116.0     136045   
...             ...     ...      ...        ...        ...        ...   
1444398       99767  390677  1108747       1362    29930.0     433605   
1444400       99769  388623   764858       1362    29930.0     993050   
1444401       99770  386568  1347417       1362    29930.0     398889   
1444404       99773  388626  1414549       1362    29930.0     590843   
1444405       99774  390678   221139       1362    29930.0    1083421   

         id.resp_p  proto   duration  orig_bytes  resp_bytes  conn_state  \
3          49560.0      1  